# Régression linéaire multiple

In [1]:
# Import des librairies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

---
## Les appartements

### Les données

In [2]:
# le dataset
df = pd.read_csv('apparts.csv')
df.head()

,taille_en_pieds_carre,nb_chambres,prix
0,2104,3,399900
1,1600,3,329900
2,2400,3,369000
3,1416,2,232000
4,3000,4,539900


In [3]:
# conversion en m²
df['taille_m2'] = 0.092903 * df['taille_en_pieds_carre']
df.head()

,taille_en_pieds_carre,nb_chambres,prix,taille_m2
0,2104,3,399900,195.467912
1,1600,3,329900,148.644800
2,2400,3,369000,222.967200
3,1416,2,232000,131.550648
4,3000,4,539900,278.709000


In [4]:
# matrice X
X = df[['taille_m2', 'nb_chambres']]
X.head()

,taille_m2,nb_chambres
0,195.467912,3
1,148.644800,3
2,222.967200,3
3,131.550648,2
4,278.709000,4


In [11]:
# vecteur y
y = df[['prix']]
y.head()

,prix
0,399900
1,329900
2,369000
3,232000
4,539900


In [12]:
# vérifier les tailles
X.shape, y.shape

((47, 2), (47, 1))

### Création des échantillons de test et d'entraînement

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=7)

In [53]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((40, 2), (7, 2), (40, 1), (7, 1))

### Construction du modèle

#### Scikit-learn

In [54]:
# le modèle
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [55]:
# évaluation R2
sc_train = lr.score(X_train, y_train)
sc_test = lr.score(X_test, y_test)
print(f"{sc_train=} et {sc_test=}")

sc_train=0.7000135219920072 et sc_test=0.7710012181258208


In [56]:
# évaluation RMSE
from sklearn.metrics import mean_squared_error
mse_train = mean_squared_error(y_train, lr.predict(X_train), squared=False)
mse_test = mean_squared_error(y_test, lr.predict(X_test), squared=False)
print(f"{mse_train=} et {mse_test=}")

mse_train=64765.844805476794 et mse_test=65923.29003484217


In [57]:
# prédictions
pred = pd.DataFrame()
pred['y_true'] = y_test
pred['y_pred'] = lr.predict(X_test)
#pred['erreur_absolue'] = abs(pred['y_true'] - pred['y_pred'])
pred

,y_true,y_pred
44,179900,212061.517554
4,539900,460459.869767
17,199900,243503.598950
31,169900,252136.500484
7,198999,269179.838211
22,242900,292974.049462
15,449900,366358.469337


#### Statsmodels

In [58]:
# construction du modèle
from statsmodels.api import OLS, add_constant
lr2 = OLS(y_train, add_constant(X_train))
res = lr2.fit()

In [60]:
# évaluation du modèle
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   prix   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.684
Method:                 Least Squares   F-statistic:                     43.17
Date:                Fri, 19 Nov 2021   Prob (F-statistic):           2.12e-10
Time:                        15:26:51   Log-Likelihood:                -499.90
No. Observations:                  40   AIC:                             1006.
Df Residuals:                      37   BIC:                             1011.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        1.379e+05   5.12e+04      2.695      0.011    3.42e+04    2.42e+05
taille_m2    1446.9993    168.335      8.596      0.000    1105.921    1788.077
nb_chambres -2.018e+04   1.77e+04     -1.138      0.263   -5.61e+04    1.58e+04
==============================================================================
Omnibus:                        4.318   Durbin-Watson:                   2.296
Prob(Omnibus):                  0.115   Jarque-Bera (JB):                3.040
Skew:                           0.611   Prob(JB):                        0.219
Kurtosis:                       3.574   Cond. No.                     1.03e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.03e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [62]:
# Nouvelles prédictions
res.predict(add_constant(X_test))

44    212061.517554
4     460459.869767
17    243503.598950
31    252136.500484
7     269179.838211
22    292974.049462
15    366358.469337
dtype: float64

### Exo

Refaire la régression en standardisant les variables explicatives. Soit "à la main", soit avec le StandardScaler de `sklearn`

---
## Les entreprises

### Les données

In [68]:
# le dataset
df = pd.read_csv('entreprises.csv')
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [84]:
# X et y
y = df.Profit
X = df.drop('Profit', axis=1)
X.shape, y.shape

((50, 4), (50,))

### Gestion des variables catégoriques

In [85]:
# Répartition
df.State.value_counts()

New York      17
California    17
Florida       16
Name: State, dtype: int64

In [87]:
X = df.drop('Profit', axis=1)

# avec pd.get_dummies
dummy = pd.get_dummies(X['State'], drop_first=True)
# on supprime une des indicatrices avec drop_first pour éviter d'avoir des variables colinéaires :
# si on les garde toutes, on a "California + Florida + New York = 1", or on en veut pas
X = pd.concat([X.drop('State', axis=1), dummy], axis=1)
X.head()

,R&D Spend,Administration,Marketing Spend,Florida,New York
0,165349.20,136897.80,471784.10,0,1
1,162597.70,151377.59,443898.53,0,0
2,153441.51,101145.55,407934.54,1,0
3,144372.41,118671.85,383199.62,0,1
4,142107.34,91391.77,366168.42,1,0


### Le modèle

In [88]:
from statsmodels.api import OLS, add_constant
lr = OLS(y, add_constant(X))
res = lr.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Fri, 19 Nov 2021   Prob (F-statistic):           1.34e-27
Time:                        16:45:19   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            5.013e+04   6884.820      7.281      0.000    3.62e+04     6.4e+04
R&D Spend           0.8060      0.046     17.369      0.000       0.712       0.900
Administration     -0.0270      0.052     -0.517      0.608      -0.132       0.078
Marketing Spend     0.0270      0.017      1.574      0.123      -0.008       0.062
Florida           198.7888   3371.007      0.059      0.953   -6595.030    6992.607
New York          -41.8870   3256.039     -0.013      0.990   -6604.003    6520.229
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""